<a href="https://colab.research.google.com/github/harshithamadarapu/Team16_Hinglish-Auto-suggestions/blob/main/NLPdemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
from datasets import Dataset
import pandas as pd
from transformers import BertTokenizer, BertForMaskedLM, BertConfig, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from gensim.models import Word2Vec

# Load the data
train_data = pd.read_csv('/content/preprocessedTrain.csv')
val_data = pd.read_csv('/content/preprocessedValidation.csv')

# Handle missing values
train_data['phrases'] = train_data['phrases'].fillna('')
val_data['phrases'] = val_data['phrases'].fillna('')

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenize the data
def tokenize_data(data):
    return data['phrases'].apply(
        lambda x: tokenizer.encode(x, add_special_tokens=True) if isinstance(x, str) else []
    )

train_tokens = tokenize_data(train_data)
val_tokens = tokenize_data(val_data)

# Prepare data for Word2Vec
def prepare_word2vec_data(tokens):
    return [tokenizer.convert_ids_to_tokens(token) for token in tokens]

train_word2vec_data = prepare_word2vec_data(train_tokens)
val_word2vec_data = prepare_word2vec_data(val_tokens)

# Train Word2Vec model
word2vec_model = Word2Vec(
    train_word2vec_data,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4
)

# Prepare data for BERT
def prepare_bert_data(tokens):
    input_ids = tokens.tolist()
    attention_masks = [[1] * len(token) for token in input_ids]
    return {'input_ids': input_ids, 'attention_mask': attention_masks}

# Convert tokenized data into a Dataset object
def create_dataset(tokens):
    bert_data = prepare_bert_data(tokens)
    return Dataset.from_dict(bert_data)

train_dataset = create_dataset(train_tokens)
val_dataset = create_dataset(val_tokens)

# Data collator for dynamic padding and masking
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    save_strategy="epoch",
    evaluation_strategy="epoch",
)

from transformers import BertForMaskedLM, BertConfig

# Initialize BERT configuration and model
model_config = BertConfig.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased', config=model_config)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.535900,nan


Epoch,Training Loss,Validation Loss
1,1.535900,nan
2,1.238800,3.850853
3,1.104900,nan


TrainOutput(global_step=70701, training_loss=1.4139561602621509, metrics={'train_runtime': 10859.348, 'train_samples_per_second': 52.084, 'train_steps_per_second': 6.511, 'total_flos': 7999478487796950.0, 'train_loss': 1.4139561602621509, 'epoch': 3.0})

In [5]:
import torch
from transformers import BertTokenizer, BertForMaskedLM

In [7]:
# Save the trained model and tokenizer
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.txt',
 './results/added_tokens.json')

In [8]:
# Load the model and tokenizer from the saved directory
tokenizer = BertTokenizer.from_pretrained('./results')
model = BertForMaskedLM.from_pretrained('./results')

In [9]:
# Function to predict the next words based on the input sentence
def predict_next_words(input_sentence, top_k=5):
    # Tokenize the input sentence
    input_ids = tokenizer.encode(input_sentence, return_tensors='pt')

    # Mask the last token for prediction
    masked_input_ids = input_ids.clone()
    masked_input_ids[0, -1] = tokenizer.mask_token_id  # Mask the last word

    # Get the attention mask
    attention_mask = (masked_input_ids != tokenizer.pad_token_id).long()

    # Get the model's prediction for the masked token
    with torch.no_grad():
        outputs = model(masked_input_ids, attention_mask=attention_mask)
        predictions = outputs.logits

    # Get the top k predicted tokens for the masked position
    predicted_ids = torch.topk(predictions[0, -1], top_k).indices.tolist()

    # Decode the predicted token ids to words
    predicted_words = tokenizer.decode(predicted_ids, skip_special_tokens=True)

    return predicted_words

In [12]:
# Take sentence input from the user
user_input = input("Enter a sentence: ")

# Predict the next 5 words
predicted_words = predict_next_words(user_input)

# Display the predicted words
print("Predicted next 5 words:", predicted_words)

Enter a sentence: uske
Predicted next 5 words: haine logo meinya
